In [2]:
import sys, platform, shutil, importlib, subprocess, json

def chk(name, want_prefix=None):
    try:
        m = importlib.import_module(name)
        ver = getattr(m, "__version__", "n/a")
        ok = (want_prefix is None) or str(ver).startswith(want_prefix)
        return {"pkg": name, "version": ver, "ok": ok, "want": want_prefix}
    except Exception as e:
        return {"pkg": name, "version": f"ERR: {e}", "ok": False, "want": want_prefix}

print("Python:", sys.version)
print("OS:", platform.platform())
print("Arch:", platform.machine())
print("which python:", shutil.which("python3"))
print("which ffmpeg:", shutil.which("ffmpeg"))

checks = [
    ("torch", "1.6.0"),
    ("torchvision", "0.7.0"),
    ("torchaudio", "0.6.0"),
    ("numpy", "1.19.5"),
    ("scipy", "1.5."),
    ("librosa", "0.8.1"),
    ("transformers", "4.12.5"),
    ("tokenizers", "0.10.3"),
    ("huggingface_hub", "0.4.0"),
    ("sentencepiece", "0.1."),
    ("sentence_transformers", "2.2."),
    ("fairseq", "0.10.2"),
    ("pororo", "0.4.2"),
]

report = [chk(n, p) for n,p in checks]
print(json.dumps(report, indent=2, ensure_ascii=False))
assert all(r["ok"] for r in report), "⚠️ 위 리스트 중 버전이 다른 항목이 있습니다. 그래도 다음 셀을 실행해보고 문제시 해당 항목만 재설치하세요."


Python: 3.8.10 (default, Mar 18 2025, 20:04:55) 
[GCC 9.4.0]
OS: Linux-6.10.14-linuxkit-x86_64-with-glibc2.29
Arch: x86_64
which python: /usr/bin/python3
which ffmpeg: /usr/bin/ffmpeg


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[
  {
    "pkg": "torch",
    "version": "1.6.0+cpu",
    "ok": true,
    "want": "1.6.0"
  },
  {
    "pkg": "torchvision",
    "version": "0.7.0+cpu",
    "ok": true,
    "want": "0.7.0"
  },
  {
    "pkg": "torchaudio",
    "version": "0.6.0",
    "ok": true,
    "want": "0.6.0"
  },
  {
    "pkg": "numpy",
    "version": "1.19.5",
    "ok": true,
    "want": "1.19.5"
  },
  {
    "pkg": "scipy",
    "version": "1.5.4",
    "ok": true,
    "want": "1.5."
  },
  {
    "pkg": "librosa",
    "version": "0.8.1",
    "ok": true,
    "want": "0.8.1"
  },
  {
    "pkg": "transformers",
    "version": "4.12.5",
    "ok": true,
    "want": "4.12.5"
  },
  {
    "pkg": "tokenizers",
    "version": "0.10.3",
    "ok": true,
    "want": "0.10.3"
  },
  {
    "pkg": "huggingface_hub",
    "version": "0.4.0",
    "ok": true,
    "want": "0.4.0"
  },
  {
    "pkg": "sentencepiece",
    "version": "0.1.99",
    "ok": true,
    "want": "0.1."
  },
  {
    "pkg": "sentence_transformers",
    "version

In [4]:
import torchaudio, os

# 가능한 경우 sox_io, 아니면 soundfile 백엔드 사용
try:
    torchaudio.set_audio_backend("sox_io")
except Exception:
    torchaudio.set_audio_backend("soundfile")

print("torchaudio backend:", torchaudio.get_audio_backend())

# 데이터 폴더(있으면) 내 파일 하나 탐색
cand_dirs = [
    "/work/data/KsponSpeech_eval/eval_clean",
    "data",
    "."
]
wav_sample = None
for d in cand_dirs:
    if os.path.isdir(d):
        for fn in os.listdir(d):
            if fn.lower().endswith(".wav"):
                wav_sample = os.path.join(d, fn)
                break
    if wav_sample: break

print("sample wav:", wav_sample if wav_sample else "(not found)")
if wav_sample:
    info = torchaudio.info(wav_sample)
    print(info)


torchaudio backend: sox_io
sample wav: /work/data/KsponSpeech_eval/eval_clean/KsponSpeech_E02927.wav


In [5]:
try:
    import wav2letter, importlib, inspect
    from wav2letter import decoder, criterion
    print("wav2letter OK:", wav2letter.__file__)
    print("decoder  OK:", inspect.getsourcefile(decoder) or "(built-in)")
    print("criterion OK:", inspect.getsourcefile(criterion) or "(built-in)")
except Exception as e:
    raise SystemExit(f"❌ wav2letter import 실패: {e}\n"
                     f"컨테이너에서 pororo/asr-install.sh 실행이 제대로 끝났는지 확인하세요.")


AttributeError: 'tuple' object has no attribute 'tb_frame'